In [ ]:
#!accelerate launch --mixed_precision fp16 ./train_accel.py --config_name EXP_02

In [ ]:
# !CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V1

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V2

In [ ]:
# !CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V0

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V1

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V2

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name MATGRAPH

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name MATGRAPHV2

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V3

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_EMBED_V5

In [ ]:
#!CUDA_VISIBLE_DEVICES=0 python train.py --config_name BASELINE_HF_V5

In [ ]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V6

In [ ]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V7

In [ ]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V8

In [ ]:
#!CUDA_VISIBLE_DEVICES=1 python train.py --config_name BASELINE_HF_V9

In [ ]:
#!CUDA_VISIBLE_DEVICES=1 python eval.py --config_name BASELINE_HF_V8FTEVAL

In [ ]:
def set_gpu_environ():
    """Sets CUDA_VISIBLE_DEVICES to those under minimal memory load.
    Meant to be used in notebooks only.
    """
    import os
    import subprocess
    query = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.used', '--format=csv']).decode().split('\n')[1:-1]
    utilization = [int(x.replace(" MiB", "")) for x in query]
    free = [i for i in range(len(utilization)) if utilization[i] == min(utilization)]
    set_visible = ",".join([str(i) for i in free])
    os.environ["CUDA_VISIBLE_DEVICES"] = set_visible
set_gpu_environ()

In [ ]:
import os
import random
import numpy as np
import torch
import config

def seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    

def train(cfg):
    seed()
    custom_model = cfg.MODEL_NAME()
    if cfg.MODEL_WTS:
        print(f"Loading model weights from {cfg.MODEL_WTS}")
        custom_model.load_state_dict(torch.load(cfg.MODEL_WTS))
    opt = cfg.OPT(
        custom_model.parameters(), lr=cfg.LR, weight_decay=cfg.WD
    )
    loss_func = cfg.LOSS_FUNC()
    scheduler = cfg.SCHEDULER(
        opt,
        num_warmup_steps=cfg.WARM_UP_PCT * cfg.EPOCHS,
        num_training_steps=cfg.EPOCHS,
    )



    cfg.FIT_FUNC(
        epochs=cfg.EPOCHS,
        model=custom_model,
        loss_fn=loss_func,
        opt=opt,
        metric=cfg.METRIC,
        config = cfg,
        folder=cfg.FOLDER/cfg.EXP_NAME,
        exp_name=f"{cfg.EXP_NAME}",
        device=cfg.DEVICE,
        sched=scheduler,
    )
    
def main(config_name):
    configs = eval(f"config.{config_name}")
    print(f"Training with config: {configs.__dict__}")
    os.makedirs(configs.FOLDER/configs.EXP_NAME)
    train(configs)

In [ ]:
main('BASELINE_graph_V9')

In [ ]:
torch.cuda.current_device()

In [ ]:
k

In [ ]:
k